In [58]:
import pandas as pd
import numpy as np

In [69]:
import utils

In [59]:
seeds = [0, 25, 50, 75, 100]
models = ['A2C', 'PPO', 'DDPG', 'SAC']
tickers = ['SPY', 'TLT', 'GLD', 'FXY']  # Asset names corresponding to asset_0, asset_1, asset_2, asset_3


## portfolio values

In [60]:
for model in models:    
    data = pd.DataFrame()
    
    # Load all seed data (portfolio values are already cumulative from evaluation mechanism)
    for seed in seeds:
        portfolio_df = pd.read_csv(f'tables/{model.lower()}_multi_seed/{model}_seed_{seed}_portfolio.csv')
        portfolio_df['date'] = pd.to_datetime(portfolio_df['date'])
        
        # Extract iteration, train dates, test dates from metadata columns
        data[f'portfolio_values_{seed}'] = portfolio_df.set_index('date')['portfolio_value']

    # Calculate mean and standard deviation across seeds
    data[f'{model.lower()}_mean'] = data.mean(axis=1, numeric_only=True)
    data[f'{model.lower()}_sd'] = data.std(axis=1, numeric_only=True)
    data.to_csv(f'tables/{model.lower()}_multi_seed/{model}_cumulative_period_portfolio_values.csv')
    
    print(f"✓ {model} cumulative portfolio values aggregated from {len(seeds)} seeds")

✓ A2C cumulative portfolio values aggregated from 5 seeds
✓ PPO cumulative portfolio values aggregated from 5 seeds
✓ DDPG cumulative portfolio values aggregated from 5 seeds
✓ SAC cumulative portfolio values aggregated from 5 seeds


## weights

In [70]:
for model in models:
    # Dictionary to store all seed data
    all_seeds_data = {}
    
    for seed in seeds:
        weights_df = pd.read_csv(f'tables/{model.lower()}_multi_seed/{model}_seed_{seed}_weights.csv')
        weights_df['date'] = pd.to_datetime(weights_df['date'])
        weights_df = weights_df.set_index('date')
        all_seeds_data[seed] = weights_df
    
    # Get asset columns (exclude metadata columns)
    asset_cols = [col for col in all_seeds_data[seeds[0]].columns if col.startswith('asset_')]
    
    # Create aggregated dataframe
    aggregated = pd.DataFrame()
    aggregated.index.name = 'date'
    
    # For each asset, calculate mean and std across seeds
    for i, asset in enumerate(asset_cols):
        ticker = tickers[i] if i < len(tickers) else f'asset_{i}'  # Map asset_i to ticker name
        asset_data = pd.concat([all_seeds_data[seed][asset] for seed in seeds], axis=1)
        aggregated[f'{ticker}_mean'] = asset_data.mean(axis=1)
        aggregated[f'{ticker}_sd'] = asset_data.std(axis=1)

    # calculate diversification ratio
    aggregated = utils.calculate_diversification_ratio(aggregated, asset_weight_cols=['SPY_mean', 'TLT_mean', 'GLD_mean', 'FXY_mean'])
    
    # Reset index to make date a column
    aggregated = aggregated.reset_index()
    aggregated.to_csv(f'tables/{model.lower()}_multi_seed/{model}_aggregated_weights.csv', index=False)
    
    print(f"✓ {model} weights aggregated from {len(seeds)} seeds (date x assets: {', '.join(tickers)})")


✓ A2C weights aggregated from 5 seeds (date x assets: SPY, TLT, GLD, FXY)
✓ PPO weights aggregated from 5 seeds (date x assets: SPY, TLT, GLD, FXY)
✓ DDPG weights aggregated from 5 seeds (date x assets: SPY, TLT, GLD, FXY)
✓ SAC weights aggregated from 5 seeds (date x assets: SPY, TLT, GLD, FXY)


## turnovers

In [62]:
for model in models:
    # Dictionary to store all seed data
    all_seeds_data = {}
    
    for seed in seeds:
        turnover_df = pd.read_csv(f'tables/{model.lower()}_multi_seed/{model}_seed_{seed}_turnover.csv')
        turnover_df['date'] = pd.to_datetime(turnover_df['date'])
        turnover_df = turnover_df.set_index('date')
        all_seeds_data[seed] = turnover_df
    
    # Create aggregated dataframe
    aggregated = pd.DataFrame()
    aggregated.index.name = 'date'
    
    # Calculate mean and std of turnover across seeds
    turnover_data = pd.concat([all_seeds_data[seed]['turnover'] for seed in seeds], axis=1)
    aggregated['turnover_mean'] = turnover_data.mean(axis=1)
    aggregated['turnover_sd'] = turnover_data.std(axis=1)
    
    # Reset index to make date a column
    aggregated = aggregated.reset_index()
    aggregated.to_csv(f'tables/{model.lower()}_multi_seed/{model}_aggregated_turnover.csv', index=False)
    
    print(f"✓ {model} turnover aggregated from {len(seeds)} seeds (date)")


✓ A2C turnover aggregated from 5 seeds (date)
✓ PPO turnover aggregated from 5 seeds (date)
✓ DDPG turnover aggregated from 5 seeds (date)
✓ SAC turnover aggregated from 5 seeds (date)


## transaction costs

In [63]:
for model in models:
    # Dictionary to store all seed data
    all_seeds_data = {}
    
    for seed in seeds:
        costs_df = pd.read_csv(f'tables/{model.lower()}_multi_seed/{model}_seed_{seed}_transaction_costs.csv')
        costs_df['date'] = pd.to_datetime(costs_df['date'])
        costs_df = costs_df.set_index('date')
        all_seeds_data[seed] = costs_df
    
    # Create aggregated dataframe
    aggregated = pd.DataFrame()
    aggregated.index.name = 'date'
    
    # Calculate mean and std of transaction costs across seeds
    costs_data = pd.concat([all_seeds_data[seed]['transaction_cost'] for seed in seeds], axis=1)
    aggregated['transaction_cost_mean'] = costs_data.mean(axis=1)
    aggregated['transaction_cost_sd'] = costs_data.std(axis=1)
    
    # Reset index to make date a column
    aggregated = aggregated.reset_index()
    aggregated.to_csv(f'tables/{model.lower()}_multi_seed/{model}_aggregated_transaction_costs.csv', index=False)
    
    print(f"✓ {model} transaction costs aggregated from {len(seeds)} seeds (date)")


✓ A2C transaction costs aggregated from 5 seeds (date)
✓ PPO transaction costs aggregated from 5 seeds (date)
✓ DDPG transaction costs aggregated from 5 seeds (date)
✓ SAC transaction costs aggregated from 5 seeds (date)
